<a href="https://colab.research.google.com/github/iragca/DS313/blob/main/notebooks/Phase2-Group3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 2: Database Setup and Data Migration
**DMA / ADBMS Final Project**

Members:


*   Chris Andrei Irag*
*   Hernel Juanico
*   Keith Laspoña
*   Airyll Sanchez
*   Kobe Marco Olaguir
*   Ruszed Jy Ayad

University of Science and Technology of Southern Philippines

*irag.chrisandrei@mailbox.org

## Utilizing AWS Postgres Relational Database to store data from AniList's API

We chose **AWS Postgres Relational Database** for the **AniList API** due to its scalability, performance optimization, and efficient handling of structured data, making it ideal for managing live anime data retrieved from the API.

It supports relational data structures to organize and query anime datasets effectively, integrates seamlessly with analytical tools like Python and Tableau, and provides robust security features, including encryption and compliance with data protection standards.

AWS RDS also ensures high availability through Multi-AZ deployments and offers a cost-effective, pay-as-you-go model, making it a reliable solution for storing and analyzing large datasets to derive meaningful insights for anime enthusiasts and developers.

Before starting, we need to import the following libraries:

*   `os`: Provides tools for managing files, directories, and system operations.
*   `pandas`: Used for transforming and viewing tabular data.
*   `psycopg2`: A PostgreSQL database adapter for Python.
*   `polars`: High-performance data manipulation library.
*   `requests`: Simplifies API requests and web data handling.
*   `dotenv`: Manages environment variables securely.

In [ ]:
try:
  import os
  import psycopg2

  import polars as pl
  import pandas as pd
  import requests

  import time
  from datetime import datetime

  from dotenv import load_dotenv; load_dotenv()
except ModuleNotFoundError:
  !pip install python-dotenv
  from dotenv import load_dotenv; load_dotenv()

  print("""
  Please restart the kernel and upload your .env file if you haven't already.
  """)

This code imports essential libraries for managing data, interacting with a PostgreSQL database, handling API requests, and working with time and environment variables. It includes **os** for system operations, **psycopg2** for database connectivity, **polars** and **pandas** for data manipulation, **requests** for HTTP requests, and time and **datetime** for time-related tasks.

The **dotenv** library is used to load environment variables securely from a .env file. If `python-dotenv` is not installed, the code installs it using `pip` and prompts the user to restart the kernel and upload the `.env` file to ensure the environment is properly set up.

Documentation:
*   https://www.psycopg.org/docs/
*   https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_ConnectToPostgreSQLInstance.html
*   https://github.com/aws/aws-advanced-python-wrapper


## Setting Up PostgreSQL Connection

This code serves as a configuration setup for establishing a connection to a PostgreSQL database hosted on **Amazon RDS (Relational Database Service).** Each parameter is essential to successfully connect and interact with the database.

**Connection Parameters**

*   `PGEND_POINT`: The endpoint of the PostgreSQL database to which the connection will be made.

*   `PGDATABASE_NAME`: The name of the database within the PostgreSQL server to connect to.

*   `PGPORT`: The port through which the database connection will be established.

*   `PGUSER_NAME`: The username used to authenticate with the PostgreSQL database.

*   `PGPASSWORD`: The password associated with the user account for connecting to the PostgreSQL database.

In [ ]:
  PGEND_POINT = os.environ["PGEND_POINT"]
  PGDATABASE_NAME = os.environ["PGDATABASE_NAME"]
  PGPORT = os.environ["PGPORT"]
  PGUSER_NAME = os.environ["PGUSER_NAME"]
  PGPASSWORD = os.environ["PGPASSWORD"]

This code defines a function named `connect()` that establishes a connection to a PostgreSQL database and returns a connection object and a cursor object.

In [ ]:
def connect():

    # Set up a connection to the postgres server.
    conn_string = f"host={PGEND_POINT} port={PGPORT} dbname={PGDATABASE_NAME} user={PGUSER_NAME} password={PGPASSWORD}"

    conn = psycopg2.connect(conn_string)
    print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tConnected!')

    # Create a cursor object
    cursor = conn.cursor()

    return conn, cursor

The `disconnect()` function to disconnects a connection to a PostgreSQL database. Once disconnected, it will display a message indicating a successful disconnection.

In [ ]:
def disconnect():
    conn.close()
    print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tDisconnected.')

## Schema

This schema defines the structure, data types, and constraints for the database.

![Schema](https://github.com/iragca/random-uploads/blob/main/DS313/Final%20Project/anime_schema.png?raw=true)

Attribute descriptions

**ANIME**

| **Column**         | **Description**                                                                                   |
|---------------------|---------------------------------------------------------------------------------------------------|
| `id`               | This is the **primary key** for the table, ensuring that each entry is unique and identifiable. It is an integer type that auto-increments with each added anime. |
| `genres`           | This column lists the genres of the anime, allowing for categorization based on style, theme, or audience. |
| `season`           | Specifies the season in which the anime was released (e.g., **FALL**, **WINTER**, **SPRING**, **SUMMER**). This classification aids in organizing content based on airing times. |
| `seasonYear`       | Indicates the year of release for the anime, providing chronological context.                     |
| `title.english`    | Holds the title of the anime in **English**, which can be utilized for international audiences and database queries. |
| `title.romaji`     | Stores the title in romaji, making it accessible for users who read **Latin** alphabets but may not know Japanese characters. |
| `title.native`     | Contains the **original title** of the anime as presented in **Japanese**, maintaining the authenticity of the title. |

**REVIEW**

| **Column**         | **Description**                                                                                   |
|---------------------|---------------------------------------------------------------------------------------------------|
| `id`               | This is the **primary key** for the table, ensuring that each entry is unique and identifiable. It is an integer type that auto-increments with each added review. |
| `mediaId`           | The media ID for an Anime or Manga.  |
| `rating`           | Total number of upvotes of the user's review |
| `ratingAmount`       | Total number of upvotes/downvotes of the user's review.                    |
| `score`    | Holds the title of the anime in **English**, which can be utilized for international audiences and database queries. |
| `body`     | The full body of the user's review. |
| `summary`     | A short summary and tagline of the user's review. |


In [ ]:
try:
  conn, cursor = connect()

  cursor.execute("""
  DROP TABLE IF EXISTS ANIME;
  """)

  cursor.execute("""
  DROP TABLE IF EXISTS REVIEW;
  """)

  cursor.execute("""
  CREATE TABLE ANIME (
    id INTEGER,
    genres VARCHAR(255),
    season VARCHAR(6),
    seasonYear INTEGER,
    "title.english" VARCHAR(512),
    "title.native" VARCHAR(512),
    "title.romaji" VARCHAR(512),

    PRIMARY KEY (id),
    CHECK (season IN ('FALL', 'WINTER', 'SPRING', 'SUMMER'))
  );
  """)


  cursor.execute("""
  CREATE TABLE REVIEW (
    id INTEGER,
    mediaId INTEGER,
    rating SMALLINT,
    ratingAmount SMALLINT,
    score SMALLINT,
    body VARCHAR(100000),
    summary VARCHAR(1000),

    PRIMARY KEY (id)
  );
  """)
  conn.commit()

except InterfaceError as e:
  print(e)

finally:
  disconnect()

2024-11-23 08:03:56	Connected!
2024-11-23 08:03:57	Disconnected.


The code interacts with a PostgreSQL database to list all tables within the **public** schema. This utilizes the `cursor.execute()` method, which executes an SQL Query that selects the table_name column from the *information_schema.tables* system table, filtering for tables in the **public** schema and ordering them alphabetically. The `cursor.fetchall()` method then retrieves all the results and displays them in in the output.

In [ ]:
try:
  conn, cursor = connect()
  cursor.execute("""
      SELECT table_name
      FROM information_schema.tables
      WHERE table_schema = 'public'  -- To filter tables in the 'public' schema (default)
      ORDER BY table_name;
  """)

  fetched = cursor.fetchall()
  print(fetched)

except InterfaceError as e:
  print(e)

finally:
  disconnect()

2024-11-23 08:04:41	Connected!
[('anime',), ('review',)]
2024-11-23 08:04:42	Disconnected.


A display all and insert data helper functions.

In [ ]:
def display_all(table: str, conn = conn):
    conn, cursor = connect()
    display(
        pl.read_database(f'SELECT * FROM {table}', conn)
    )
    disconnect()

In [ ]:
def insert_data(table: str, data: pd.DataFrame):
    try:

      conn, cursor = connect()
      start_time = time.time()

      print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tInserting...')
      vals = '%s, ' * (len(data.columns)-1)
      cursor.executemany(f"INSERT INTO {table} VALUES ({vals}%s)", list(data.itertuples(index=False, name=None)))
      end_time = time.time()
      elapsed_time = end_time - start_time
      print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tOperation took {elapsed_time:.4f} seconds.')
      conn.commit()

    except Exception as e:
      conn.rollback()
      print(e)
      print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tOperation failed.')

    else:
      print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\tSuccessful operation.')

    finally:
      disconnect()


## Generate and Insert mock data

This code imports data from a specified GitHub repository. It reads the data into a pandas DataFrame named mock_data.

In [ ]:
# mock data from google
mock_data = pd.read_csv("https://github.com/iragca/random-uploads/raw/refs/heads/main/DS313/Final%20Project/anime.csv")

This code is designed to perform data cleaning operations. First, the season column is converted to uppercase to ensure consistency across the data. Next, only the relevant columns— 'id', 'genres', 'season', 'seasonYear', 'title.english', 'title.native', and 'title.romaji'—are selected for further analysis. Finally, rows containing any missing values are removed using the dropna(axis=0)function.

In [ ]:
# Cleaning data.
mock_data['season'] = mock_data['season'].str.upper()
mock_data = mock_data[['id', 'genres', 'season', 'seasonYear', 'title.english', 'title.native', 'title.romaji']]
mock_data = mock_data.dropna(axis=0)

In [ ]:
mock_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7740 entries, 0 to 23992
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             7740 non-null   int64 
 1   genres         7740 non-null   object
 2   season         7740 non-null   object
 3   seasonYear     7740 non-null   int64 
 4   title.english  7740 non-null   object
 5   title.native   7740 non-null   object
 6   title.romaji   7740 non-null   object
dtypes: int64(2), object(5)
memory usage: 483.8+ KB


In [ ]:
mock_data.head(5)

,id,genres,season,seasonYear,title.english,title.native,title.romaji
0,5114,"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...",SPRING,2009,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,['Hagane no Renkinjutsushi: Fullmetal Alchemis...
1,11061,"['Action', 'Adventure', 'Fantasy', 'Shounen']",FALL,2011,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,['HxH (2011)']
2,9253,"['Drama', 'Psychological', 'Sci-Fi', 'Suspense...",SPRING,2011,Steins;Gate,STEINS;GATE,[]
3,21,"['Action', 'Adventure', 'Fantasy', 'Shounen']",FALL,1999,One Piece,ONE PIECE,['OP']
4,1535,"['Psychological', 'Shounen', 'Supernatural', '...",FALL,2006,Death Note,デスノート,['DN']


The following code inserts a random sample of 20 rows from the mock_data DataFrame into a table named "ANIME" in a database. First it connects to the database, inserts the data, and then disconnects.

In [ ]:
insert_data(table='ANIME', data=mock_data.sample(20))

2024-11-23 08:05:22	Connected!
2024-11-23 08:05:22	Inserting...
2024-11-23 08:05:26	Operation took 4.4675 seconds.
2024-11-23 08:05:27	Successful operation.
2024-11-23 08:05:27	Disconnected.


In [ ]:
display_all("ANIME")

2024-11-23 08:05:51	Connected!


id,genres,season,seasonyear,title.english,title.native,title.romaji
i64,str,str,i64,str,str,str
5774,"""['Boys Love', 'Comedy', 'Drama…","""FALL""",2009,"""Hey, Class President!""","""生徒会長に忠告""","""['Hey Class President', 'High-…"
3060,"""['Action', 'Mecha', 'Sci-Fi']""","""FALL""",1983,"""Dallos""","""ダロス""","""['Darosu', 'Battle for Moon St…"
3104,"""['Action', 'Military', 'Sci-Fi…","""FALL""",2001,"""Geisters: Fractions of the Ear…","""ガイスターズ FRACTIONS OF THE EARTH""","""['Guystars: Fractions of the E…"
3372,"""['Adventure']""","""FALL""",2006,"""RGB Adventure""","""RGBアドベンチャ""","""[]"""
42627,"""['Fantasy', 'Shounen']""","""SUMMER""",2021,"""Peach Boy Riverside""","""ピーチボーイリバーサイド""","""[]"""
…,…,…,…,…,…,…
4508,"""['Comedy', 'Harem', 'Romance']""","""FALL""",2005,"""D.C.S.S: Da Capo Second Season…","""D.C. ～ダ・カーポ～ セコンドシースン Omake""","""['D.C.S.G. Omake', 'Da Capo 2n…"
34102,"""['Mystery', 'School', 'Super P…","""SPRING""",2017,"""Sagrada Reset""","""サクラダリセット""","""[]"""
51050,"""['Music', 'Space']""","""WINTER""",2020,"""Saturn""","""サターン""","""[]"""


2024-11-23 08:05:52	Disconnected.



## Fetch and Store Data

Removing all the mock data in the table.

In [ ]:
try:
  conn, cursor = connect()
  cursor.execute("""
  TRUNCATE TABLE ANIME;
  """)
  conn.commit()
except Exception as e:
  print(e)
finally:
  disconnect()

2024-11-23 08:06:06	Connected!
2024-11-23 08:06:07	Disconnected.


In [ ]:
display_all("ANIME")

2024-11-23 08:06:25	Connected!


id,genres,season,seasonyear,title.english,title.native,title.romaji
null,null,null,null,null,null,null


2024-11-23 08:06:25	Disconnected.


This code fetches anime data from the **Anilist API** using a **GraphQL query,** it specifies the desired season, year, page number, and items per page. The fetched data is then processes using the pandas library to turn it into a DataFrame, which makes it easier to analyze and manipulate. This allows for efficient and structured retriveal of anime information from the API.

Fetching the data from the AniList database through their API

In [ ]:
query = '''
query ExampleQuery($page: Int, $perPage: Int, $season: MediaSeason, $seasonYear: Int, $sort: [ReviewSort], $type: MediaType) {
      Page (page: $page, perPage: $perPage) {
        pageInfo {
          currentPage
          hasNextPage
          perPage
        }

      media(season: $season, seasonYear: $seasonYear, type: $type) {
          id
          title {
            english
            native
            romaji
          }
          genres
          season
          seasonYear
        reviews(sort: $sort) {
          edges {
            node {
              id
              mediaId
              rating
              ratingAmount
              score
              body
              summary
            }
          }
        }
      }
      }
}
'''

The rate limit for the AniList API is 30 requests per minute, with a maximum of 50 items retrieved per page. So we will do our best effort to get all the data in multiple pages and concatenate them into one pandas dataframe.

```python
'perPage': 300
```

Notice the no. of items retrieved per page is capped at 50 and the 300 items to retrieve per page request parameter is ignored.

In [ ]:
url = 'https://graphql.anilist.co'
data = []
anime_data = pd.DataFrame()

try:
  for i in range(1, 5): # max range is 30
    variables = {
      "page": i,
      "perPage": 300, # max retrieved is 50
      "seasonYear": 2024,
      "season": "FALL",
      "sort": "ID",
      "type": "ANIME",
    }
    response = requests.post(url, json={'query': query, 'variables': variables})
    response_data = response.json()
    data.append(pd.json_normalize(response_data['data']['Page']['media']))

  anime_data = pd.concat(data, axis=0)

except Exception as e:
  print(response.json()['errors'])

In [ ]:
anime_data

,id,genres,season,seasonYear,title.english,title.native,title.romaji,reviews.edges
0,107372,[Fantasy],FALL,2024,The Stories of Girls Who Couldn't Be Magicians,魔法使いになれなかった女の子の話。,Mahoutsukai ni Narenakatta Onnanoko no Hanashi.,[]
1,111314,"[Drama, Horror, Mystery, Supernatural]",FALL,2024,None,うずまき,Uzumaki,"[{'node': {'id': 25845, 'mediaId': 111314, 'ra..."
2,138522,"[Action, Drama, Fantasy]",FALL,2024,None,Thunderbolt Fantasy 東離劍遊紀4,Thunderbolt Fantasy: Touriken Yuuki 4,[]
3,141182,"[Action, Drama, Fantasy, Romance]",FALL,2024,Seirei Gensouki: Spirit Chronicles Season 2,精霊幻想記2,Seirei Gensouki 2,[]
4,150930,"[Comedy, Romance]",FALL,2024,Murai In Love,村井の恋,Murai no Koi,[]
...,...,...,...,...,...,...,...,...
7,183473,[Comedy],FALL,2024,None,ぬいストーリー2,Nui Story 2,[]
8,183651,"[Mecha, Sci-Fi]",FALL,2024,None,機動戦士ガンダム ALC ENCOUNTER,Kidou Senshi Gundam ALC ENCOUNTER,[]
9,184099,[Horror],FALL,2024,None,Agent Blue,Agent Blue,[]
10,184288,[Comedy],FALL,2024,None,カズヒホとマリーのようこそキッチン。,Kazuhiho to Marie no Youkoso Kitchen.,[]


It is observed that there are 112 rows, so there are 112 anime shows that were released in the Fall of 2024.

Inserting real data

In [ ]:
# not including the reviews
insert_data(table='ANIME', data=anime_data.loc[:, 'id':'title.romaji'])

2024-11-23 08:07:30	Connected!
2024-11-23 08:07:30	Inserting...
2024-11-23 08:07:54	Operation took 24.2764 seconds.
2024-11-23 08:07:54	Successful operation.
2024-11-23 08:07:54	Disconnected.


In [ ]:
display_all('ANIME')

2024-11-23 08:08:00	Connected!


id,genres,season,seasonyear,title.english,title.native,title.romaji
i64,str,str,i64,str,str,str
107372,"""{Fantasy}""","""FALL""",2024,"""The Stories of Girls Who Could…","""魔法使いになれなかった女の子の話。""","""Mahoutsukai ni Narenakatta Onn…"
111314,"""{Drama,Horror,Mystery,Supernat…","""FALL""",2024,null,"""うずまき""","""Uzumaki"""
138522,"""{Action,Drama,Fantasy}""","""FALL""",2024,null,"""Thunderbolt Fantasy 東離劍遊紀4""","""Thunderbolt Fantasy: Touriken …"
141182,"""{Action,Drama,Fantasy,Romance}""","""FALL""",2024,"""Seirei Gensouki: Spirit Chroni…","""精霊幻想記2""","""Seirei Gensouki 2"""
150930,"""{Comedy,Romance}""","""FALL""",2024,"""Murai In Love""","""村井の恋""","""Murai no Koi"""
…,…,…,…,…,…,…
183473,"""{Comedy}""","""FALL""",2024,null,"""ぬいストーリー2""","""Nui Story 2"""
183651,"""{Mecha,Sci-Fi}""","""FALL""",2024,null,"""機動戦士ガンダム ALC ENCOUNTER""","""Kidou Senshi Gundam ALC ENCOUN…"
184099,"""{Horror}""","""FALL""",2024,null,"""Agent Blue""","""Agent Blue"""


2024-11-23 08:08:01	Disconnected.


The code begins by initializing an empty DataFrame named 'review_data', which will be used to store review information. Then it iterates through the 'reviews.edges' field of the 'anime_data' DataFrame, processing each review. During the iteration, the code checks if the current review contains content. If so, the review is normalized into a DataFrame and concatenated to the 'review_data' DataFrame. As a result, the final 'review_data' DataFrame contains all the extracted review information.

In [ ]:
# reviews

review_data = pd.DataFrame()

for i in anime_data['reviews.edges']:
  if len(i) > 0:
    review_data = pd.concat([review_data, pd.json_normalize(i)], axis=0)

review_data

,node.id,node.mediaId,node.rating,node.ratingAmount,node.score,node.body,node.summary
0,25845,111314,101,112,50,~~~\n\n#*Originally written for my YouTube cha...,The Animation Was the Scariest Thing About Thi...
1,25846,111314,118,131,21,>#####___This review is spoiler-free.___\n\n~~...,"Scandalous, slovenly, and slapdash—a screw-up ..."
2,25847,111314,23,30,25,"***\n<img src=""https://sm.ign.com/t/ign_pk/pho...",They had us in the first quarter
3,25848,111314,24,32,17,__This review is spoilerless and anything rela...,Another Studio Tarnishes Ito's Name Once Again
4,25857,111314,49,81,10,# Review is light in spoilers\nwhat a boring s...,Uzumaki: a spiral to boredom
5,25931,111314,8,32,81,Two days after the second episode of Uzumaki r...,"Is it better to cancel an unfinished series, o..."
0,25832,166703,14,18,70,"<p align=""justify"">Okay, standard disclaimer f...",mfw the suit is mobile
0,25854,182469,223,235,100,When I knew a few weeks ago that Megumi Ishita...,A love letter to Oda's creation
1,25862,182469,19,191,64,The special begins on a recap of the Marinefor...,Ishitani would do right to sacrifice the notic...


The following code connects to a database, inserts the review_data DataFrame into the REVIEW table, and logs the progress and outcome. The messages confirm a successful connection, data insertion, the operation's duration, and the disconnection after completion.

In [ ]:
insert_data(table='REVIEW', data=review_data)

2024-11-23 08:08:52	Connected!
2024-11-23 08:08:52	Inserting...
2024-11-23 08:08:55	Operation took 2.5762 seconds.
2024-11-23 08:08:55	Successful operation.
2024-11-23 08:08:55	Disconnected.


This retrieves and displays all the data stored in the REVIEW table. It allows verification that the data insertion was successful and that the table's contents are as expected.

In [ ]:
 display_all('REVIEW')

2024-11-23 08:08:56	Connected!


id,mediaid,rating,ratingamount,score,body,summary
i64,i64,i64,i64,i64,str,str
25845,111314,101,112,50,"""~~~ #*Originally written for …","""The Animation Was the Scariest…"
25846,111314,118,131,21,""">#####___This review is spoile…","""Scandalous, slovenly, and slap…"
25847,111314,23,30,25,"""*** <img src=""https://sm.ign.c…","""They had us in the first quart…"
25848,111314,24,32,17,"""__This review is spoilerless a…","""Another Studio Tarnishes Ito's…"
25857,111314,49,81,10,"""# Review is light in spoilers …","""Uzumaki: a spiral to boredom"""
25931,111314,8,32,81,"""Two days after the second epis…","""Is it better to cancel an unfi…"
25832,166703,14,18,70,"""<p align=""justify"">Okay, stand…","""mfw the suit is mobile"""
25854,182469,223,235,100,"""When I knew a few weeks ago th…","""A love letter to Oda's creatio…"
25862,182469,19,191,64,"""The special begins on a recap …","""Ishitani would do right to sac…"


2024-11-23 08:08:57	Disconnected.
